# Quantum Kernels

<span style="color: red; font-weight: bold;">
Please replace the "?" signs by real code!
</span>

## Introduction to quantum kernels

The "Quantum kernel method" refers to any method that uses quantum computers to estimate a kernel. In this context, "kernel" will refer to the kernel matrix or individual entries therein. Recall that a feature mapping $\Phi(\vec{x})$ is a mapping from $\vec{x}\in \mathbb{R}^d$ to $\Phi(\vec{x})\in \mathbb{R}^{d'},$ where usually $d'>d$ and where the goal of this mapping is to make the categories of data separable by a hyperplane. The kernel function takes vectors in the feature-mapped space as arguments and returns their inner product, i.e. $K:\mathbb{R}^d\times\mathbb{R}^d\rightarrow \mathbb{R}$ with $K(x,y) = \langle \Phi(x)|\Phi(y)\rangle$. Classically, we are interested in feature maps for which the kernel function is easy to evaluate. This often means finding a kernel function for which the inner product in the feature-mapped space can be written in terms of the original data vectors, without having to ever construct $\Phi(x)$ and $\Phi(y)$. In the method of quantum kernels, the feature mapping is done by a quantum circuit, and the kernel is estimated using measurements on that circuit and the relative measurement probabilities.

In this notebook we will examine the depths of pre-coded encoding circuits that use substantial entanglement and compare those to depths of circuits we code by hand. This is not to advocate for one method over another. You may find that pre-coded circuits are too deep, and that the entanglement in the custom-built circuit is insufficient to be useful. Again, these are shown only to enable your exploration.

Before walking through a kernel matrix estimation in detail, let us outline the workflow using the language of Qiskit patterns.

### Step 1: Map classical inputs to a quantum problem

*   Input: Training dataset
*   Output: Abstract circuit for calculating a kernel matrix entry

Given the dataset, the starting point is to encode the data into a quantum circuit. In other words, we need to map our data into the Hilbert space of states of our quantum computer. We do this by constructing a data-dependent circuit. There are many ways of doing this, and the previous lesson outlined a number of options. 
<span style="color: blue; font-weight: bold;"> You can construct your own circuit to encode your data, or you can use a pre-made feature map like `zz_feature_map`. In this tutorial, we wil do both.</span>

Note that in order to calculate a single kernel matrix element, we will want to encode two different points, so we can estimate their inner product. A full quantum kernel workflow will of course, involve many such inner products between mapped data vectors, as well as classical machine learning methods. But the core step being iterated is the estimation of a single kernel matrix element. For this we select a data-dependent quantum circuit and map two data vectors into the feature space.

For the task of generating a kernel matrix, we are particularly interested in the probability of measuring the $|0\rangle^{\otimes N}$ state, in which all $N$ qubits are in the $|0\rangle$ state. To see this, consider that the circuit responsible for encoding and mapping of one data vector $\vec{x}_i$ can be written as $\Phi(\vec{x}_i)$, and the one responsible for encoding and mapping $\vec{x}_j$ is $\Phi(\vec{x}_j)$, and denote the mapped states

$$
|\psi(\vec{x}_i)\rangle = \Phi(\vec{x}_i)|0\rangle^{\otimes N}
$$

$$
|\psi(\vec{x}_j)\rangle = \Phi(\vec{x}_j)|0\rangle^{\otimes N}.
$$

These states *are* the mapping of the data to higher dimensions, so our desired kernel entry is the inner product

$$
\langle\psi(\vec{x}_j)|\psi(\vec{x}_i)\rangle = \langle 0 |^{\otimes N}\Phi^\dagger(\vec{x}_j)\Phi(\vec{x}_i)|0\rangle^{\otimes N}.
$$

If we operate on the default initial state $|0\rangle^{\otimes N}$ with both circuits $\Phi^\dagger(\vec{x}_j)$ and $\Phi(\vec{x}_i)$, the probability of then measuring the state $|0\rangle^{\otimes N}$ is

$$
P_0 = |\langle0|^{\otimes N}\Phi^\dagger(\vec{x}_j)\Phi(\vec{x}_i)|0\rangle^{\otimes N}|^2.
$$

This is exactly the value we want (up to $||^2$). 

<span style="color: blue; font-weight: bold;">The measurement layer of our circuit will return measurement probabilities (or so-called "quasi-probabilities", if certain error mitigation methods are used). The probability of interest is that of the zero state, $|0\rangle^{\otimes N}$.
</span>

### Step 2: Optimize problem for quantum execution

*   Input: Abstract circuit, not optimized for a particular backend
*   Output: Target circuit and observable, optimized for the selected QPU

In this step, we will use the `generate_preset_pass_manager` function from Qiskit to specify an optimization routine for our circuit with respect to the real quantum computer on which we plan to run the experiment. We set `optimization_level=3` , which means we will use the preset pass manager which provides the highest level of optimization. In this context, "optimization" refers to optimizing the implementation of the circuit on a real quantum computer. This includes considerations like selecting physical qubits to correspond to qubits in the abstract quantum circuit that will minimize gate depth, or selecting physical qubits with the lowest available error rates. This is not directly related to optimization of the machine learning problem (as in classical optimizers like COBYLA).

Depending on how you implement step 2, you may have to optimize the circuit more than once, since each pair of points involved in a matrix element produce a different circuit to be measured.

### Step 3: Execute using Qiskit Runtime Primitives

*   Input: Target circuit
*   Output: Probability distribution

Use the `Sampler` primitive from Qiskit Runtime to reconstruct a probability distribution of states yielded from sampling the circuit. Note that you may see this referred to as a "quasi-probability distribution", a term which is applicable where noise is an issue and when extra steps are introduced, such as in error mitigation. In such cases, the sum of all probabilities may not exactly equal 1; hence "quasi-probability".

Since we optimized the circuit for the backend in Step 2, we can avoid doing transpilation on the Runtime server by setting `skip_transpilation=True` and passing the optimized circuit to the `Sampler`.

### Step 4: Post-process, return result in classical format

*   Input: Probability distribution
*   Output: A single kernel matrix element, or a kernel matrix if repeating

Calculate the probability of measuring $|0\rangle^{\otimes N}$ on the quantum circuit, and populate the kernel matrix in the position corresponding to the two data vectors used. To fill out the entire kernel matrix, we need to run a quantum experiment for each entry. Once we have a kernel matrix, we can use it in many classical machine learning algorithms that accept `pre-calculated kernels`. For example: `qml_svc = SVC(kernel="precomputed")`. We can then use classical workstreams to apply our model on our testing data, and get an accuracy score. Depending on our satisfaction with our accuracy score, we may need to revisit aspects of our calculation, such as our feature map.

### Notebook outline

In this notebook we will carry out these steps several ways to make optimal use of your time on real quantum computers. 

<span style="color: blue; font-weight: bold;">
We will apply a quantum kernel method to an entire data set with relatively few features, using a simulated backend, so that we can see how the quantum workstream connects with classical machine learning methods
</span>


In [ ]:
# If you have not already, install scikit learn
#!pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from qiskit.circuit.library import unitary_overlap

## Full kernel matrix


We apply the above process to the binary classification of a full dataset. This will introduce two important components: (1) we can now implement classical machine learning in post-processing, and (2) we can obtain accuracy scores for our training.

    
### Step 1: Map classical inputs to a quantum problem

<span style="color: blue; font-weight: bold;">
Now we will import an existing dataset for our classification. This dataset consists of 128 rows (data points) and 14 features on each point. There is a 15th element that indicates the binary category of each point ($\pm 1$). 
</span>

The dataset is imported below, or you can access the dataset and view its structure [here](https://github.com/qiskit-community/prototype-quantum-kernel-training/blob/main/data/dataset_graph7.csv).

<span style="color: blue; font-weight: bold;">
We will use the first 90 data points for training, and the next 30 points for testing.
</span>


In [ ]:
#!wget https://raw.githubusercontent.com/qiskit-community/prototype-quantum-kernel-training/main/data/dataset_graph7.csv

df = pd.read_csv("?", sep=",", header=None)

# Prepare training data

train_size = ?
X_train = df.values[0:train_size, :-1]
train_labels = df.values[0:train_size, -1]

# Prepare testing data
test_size = ?
X_test = df.values[train_size : train_size + test_size, :-1]
test_labels = df.values[train_size : train_size + test_size, -1]

**Let us take a look at some samples:**

In [ ]:
X_train[0:5]

In [ ]:
train_labels[0:5]

In [ ]:
X_test[0:5]

In [ ]:
test_labels[0:5]

We will already prepare for storing multiple outputs by constructing a kernel matrix and a test matrix of appropriate dimensions.



In [ ]:
# Empty kernel matrix
num_samples = np.shape(X_train)[0]
kernel_matrix = np.full((?, ?), np.nan)
test_matrix = np.full((test_size, ?), np.nan)

In [ ]:
print(num_samples)

In [ ]:
kernel_matrix.shape

In [ ]:
test_matrix.shape

Now we create a feature map for encoding and mapping our classical data in a quantum circuit. We are free to construct our own feature map or use a pre-fabricated one. Feel free to modify the feature map below, or switch back to ZFeatureMap. But always pay attention to circuit depth. Recall that in the previous 6-qubit example the transpiled circuit depth was intractably high when using `zz_feature_map`. 

<span style="color: blue; font-weight: bold;">
As the scale and complexity of the circuit increase, the depth could rapidly increase to a point where noise overwhelms our results. Whenever you know something about your data structure that may inform what feature map structure would be most useful, it is advisable to create your own custom feature map that leverages that knowledge.
</span>


In [ ]:
from qiskit.circuit import Parameter, ParameterVector, QuantumCircuit

# Prepare feature map for computing overlap
num_features = np.shape(X_train)[1]
num_qubits = int(num_features / 2)

# To use a custom feature map use the lines below.
entangler_map = [[0, 2], [3, 4], [2, 5], [1, 4], [2, 3], [4, 6]]

fm = QuantumCircuit(num_qubits)
training_param = Parameter("θ")
feature_params = ParameterVector("x", num_qubits * 2)
fm.ry(training_param, fm.qubits)
for cz in ?:
    fm.cz(cz[0], cz[1])
for i in range(?):
    fm.rz(-2 * feature_params[2 * i + 1], i)
    fm.rx(-2 * feature_params[2 * i], i)

In [ ]:
fm.draw('mpl')

### Step 2 & 3: Optimize problem & execute using primitives

We will construct an overlap circuit, and if we were running on a real quantum computer in this example, we would optimize it for execution as before. But in this case, we intend to step over all data points and calculate the full kernel matrix. For each pair of data vectors $\vec{x}_i$ and $\vec{x}_j$, we create a different overlap circuit. 

<span style="color: blue; font-weight: bold;">
Thus we must optimize our circuit for each data point pair. So steps 2 & 3 would be done together in the multiple iterations.
</span>

We need two `for` loops, and there is the additional line at the end `kernel_matrix[x_1,x_2] = ...` to store the results of each calculation. Note that we have leveraged the symmetry of a kernel matrix to reduce the number of calculations by 1/2. 

We have also simply set the diagonal elements to 1, as they should be in the absence of noise. Depending on your implementation and required precision, you could also use the diagonal elements to estimate noise or learn about it for error mitigation purposes.

Once the kernel matrix has been fully populated, we repeat the process for the test data and populate the test\_matrix. This is really also a kernel matrix; we simply give it a different name to distinguish the two.



In [ ]:
# To use a simulator
from qiskit.primitives import StatevectorSampler

# Remember to insert your token in the QiskitRuntimeService constructor to use real quantum computers
# service = QiskitRuntimeService()
# backend = service.least_busy(
#    operational=True, simulator=False, min_num_qubits=fm.num_qubits
# )

num_shots = 10000

# Evaluate the problem using state vector-based primitives from Qiskit.
sampler = StatevectorSampler()

for x1 in range(0, train_size):
    for x2 in range(x1 + 1, train_size):
        unitary1 = fm.assign_parameters(list(X_train[x1]) + [np.pi / 2])
        unitary2 = fm.assign_parameters(list(X_train[x2]) + [np.pi / 2])

        # Create the overlap circuit
        overlap_circ = unitary_overlap(?, ?)
        overlap_circ.measure_all()

        # These lines run the qiskit sampler primitive.
        counts = (
            sampler.run([?], shots=num_shots)
            .result()[0]
            .data.meas.get_int_counts()
        )

        # Assign the probability of the 0 state to the kernel matrix, and the transposed element (since this is an inner product)
        kernel_matrix[x1, x2] = counts.get(0, 0.0) / num_shots
        kernel_matrix[?, ?] = counts.get(0, 0.0) / ?
    # Fill in on-diagonal elements with 1, again, since this is an inner-product corresponding to probability (or alter the code to check these entries and verify they yield 1)
    kernel_matrix[x1, x1] = ?

print("training done")

# Similar process to above, but for testing data.
for x1 in range(0, test_size):
    for x2 in range(0, train_size):
        unitary1 = fm.assign_parameters(list(X_test[x1]) + [np.pi / 2])
        unitary2 = fm.assign_parameters(list(X_train[x2]) + [np.pi / 2])

        # Create the overlap circuit
        overlap_circ = unitary_overlap(?, ?)
        overlap_circ.measure_all()

        counts = (
            sampler.run([?], shots=num_shots)
            .result()[0]
            .data.meas.get_int_counts()
        )

        test_matrix[?, ?] = counts.get(0, 0.0) / ?

print("test matrix done")

### Step 4: Post-process, return result in classical format

Now that we have a kernel matrix and a similarly formatted test\_matrix from quantum kernel methods, we can apply classical machine learning algorithms to make predictions about our test data and check its accuracy. We will start by importing Scikit-Learn's `sklearn.svc`, a support vector classifier (SVC). 

<span style="color: blue; font-weight: bold;">
We must specify that we want the SVC to use our precomputed kernel using `kernel = precomputed`.
</span>

In [ ]:
# import a support vector classifier from a classical ML package.
from sklearn.svm import SVC

# Specify that you want to use a pre-computed kernel matrix
qml_svc = SVC(kernel="?")

Using `SVC.fit`, we can now feed in the kernel matrix and the training labels to obtain a fit. `SVC.score` will then score our test data against that fit using our test\_matrix, and return our accuracy.



In [ ]:
# Feed in the pre-computed matrix and the labels of the training data. The classical algorithm gives you a fit.
?.fit(?, ?)

# Now use the .score to test your data, using the matrix of test data, and test labels as your inputs.
qml_score_precomputed_kernel = qml_svc.score(?, ?)
print(f"Precomputed kernel classification test score: {qml_score_precomputed_kernel}")

We see that the accuracy of our trained model was 100%. This is great, and it shows that QKE can work. 

But that is very different from quantum advantage. Classical kernels would likely have been able to solve this classification problem with 100% accuracy as well. 

<span style="color: blue; font-weight: bold;">
There is much work to be done characterizing different data types and data relationships to see where quantum kernels will be most useful in the current utility era.
</span>
    

We leave it to the learner to modify parts of this workflow and study the effectiveness of various quantum feature maps. Here are a few things to consider:

*   How robust is the accuracy? Does it hold for broad types of data or just this specific training data?
*   What structure in your data makes you suspect that a quantum feature map is useful?
*   How is the accuracy affected by increasing/decreasing the amount of training data?
*   What feature maps can you use and how do the results vary with feature maps?
*   How are the accuracy and running time affected by increasing the number of features?
*   Which trends, if any, do you expect to hold on real quantum computers?


# End of Notebook